In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
import numpy as np
import pandas as pd

import torch
from  torch import nn

In [3]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [4]:
y = pd.read_csv(labels_path, index_col='filename')

In [5]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [6]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [7]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [8]:
# data.show_batch(3, figsize=(6,6))

In [9]:
learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=MSELossFlat()
)#.to_fp16()

In [10]:
learn.path = Path('..')

In [11]:
# learn.lr_find()
# learn.recorder.plot()

In [12]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(
#     30, 
#     slice(1e-2), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=3)
#     ]
# )

In [13]:
# learn = learn.load('fastai_model')
# learn = learn.to_fp32()
# learn.save('fastai_model')
# learn.export('models/export.pkl')

In [14]:
learn = load_learner('../models', 'export.pkl')

In [15]:
# learn.show_results(rows=10)

### Bentoml Route

In [16]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

# from fastai.vision import *

@env(pip_dependencies=['pillow==6.2.2', 'fastai'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return str(result)

Overwriting pet_regression.py


In [19]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2020-01-12 20:21:28,063] INFO - BentoService bundle 'PetRegression:20200112202115_252C5F' created at: /tmp/bentoml-temp-8_nny1bp
[2020-01-12 20:21:28,344] INFO - BentoService bundle 'PetRegression:20200112202115_252C5F' created at: /home/ec2-user/bentoml/repository/PetRegression/20200112202115_252C5F


In [20]:
from bentoml import load

service = load(saved_path)

# print(service.predict(data.get(0)))

[2020-01-12 20:21:28,755] WARNING - Module `pet_regression` already loaded, using existing imported module.


In [21]:
!pip install --upgrade {saved_path}

Processing /home/ec2-user/bentoml/repository/PetRegression/20200112202115_252C5F
  Using cached https://files.pythonhosted.org/packages/a4/ad/707fb1cd011fe772b803b5c106fa4fbb5e12448fa4db2ac42d01db4859cf/BentoML-0.5.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/fd/bbbc569f98f47813c50a116b539d97b3b17a86ac7a309f83b2022d26caf2/Pillow-6.2.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f5/e4/a7025bf28f303dbda0f862c09a7f957476fa92c9271643b4061a81bb595f/fastai-1.0.60-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1a/de/f7f985018f462ceeffada7f6e609919fbcc934acd9301929cba14bc2c24a/imageio-2.6.1-py3-none-any.whl
     |████████████████████████████████| 2.4MB 14.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/90/df/88bff450f333114680698dc4aac7506ff7cab164b794461906de31998665/humanfriendly-4.18-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/ca/92

In [22]:
# !PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

In [23]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20200112202115_252C5F


In [24]:
!bentoml deployment create facial-features-detector --bento {bento_tag} --platform aws-sagemaker --api-name predict --region us-east-1

[2020-01-12 20:23:49,138] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-01-12 20:23:49,139] INFO - 

[2020-01-12 20:23:49,140] INFO -  ---> 406f2b43ea59

[2020-01-12 20:23:49,140] INFO - Step 2/11 : EXPOSE 8080
[2020-01-12 20:23:49,140] INFO - 

[2020-01-12 20:23:49,141] INFO -  ---> Using cache

[2020-01-12 20:23:49,141] INFO -  ---> 4bdc3a4eb616

[2020-01-12 20:23:49,141] INFO - Step 3/11 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential     && apt-get install -y nginx      && rm -rf /var/lib/apt/lists/*
[2020-01-12 20:23:49,141] INFO - 

[2020-01-12 20:23:49,141] INFO -  ---> Using cache

[2020-01-12 20:23:49,142] INFO -  ---> 3f2f9daae4a3

[2020-01-12 20:23:49,142] INFO - Step 4/11 : RUN conda install pip numpy scipy       && pip install gunicorn gevent
[2020-01-12 20:23:49,142] INFO - 

[2020-01-12 20:23:49,142] INFO -  ---> Using cache

[2020-01-12 20:23:49,143] INFO -  ---> 97a5e9c

In [25]:
!bentoml --verbose deployments delete facial-features-detector --force

[2020-01-12 20:17:19,918] DEBUG - Using BentoML with local Yatai server
[2020-01-12 20:17:20,100] DEBUG - Upgrading tables to the latest revision
[2020-01-12 20:17:21,026] DEBUG - AWS delete endpoint response: {'ResponseMetadata': {'RequestId': 'c10ab1bc-ce0c-48b1-9b7c-fec2b0dec6a5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c10ab1bc-ce0c-48b1-9b7c-fec2b0dec6a5', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Sun, 12 Jan 2020 20:17:20 GMT'}, 'RetryAttempts': 0}}
Successfully deleted deployment "facial-features-detector"
